<a href="https://colab.research.google.com/github/cyyeh/kaggle/blob/master/google-qa/google_qa_yesno_albert_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Environment Setup

In [0]:
# make sure colab use tf2.x
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [4]:
# install huggingface transformers
!pip install transformers

     |████████████████████████████████| 481kB 3.5MB/s 
     |████████████████████████████████| 870kB 54.7MB/s 
     |████████████████████████████████| 1.0MB 39.5MB/s 
     |████████████████████████████████| 3.1MB 45.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e7a388a37b807b498c10f061605ce94137f133882088e34add6862ce74bfc29b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import TFAlbertPreTrainedModel, TFAlbertModel, AlbertConfig
from transformers.modeling_tf_utils import get_initializer

### Setup TPU

In [6]:
# create tpu resolver and strategy
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: 10.83.87.26:8470


INFO:tensorflow:Initializing the TPU system: 10.83.87.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Load Training Dataset

In [7]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
if os.path.exists('drive/My Drive/yes_no_ans_df.pkl') and \
os.path.exists('drive/My Drive/short_ans_raw_df.pkl'):
  print("Training dataset is available!")
else:
  print("Training dataset is not found, please infer the colab notebook below to prepare training data")

Training dataset is available!


If training dataset is not found, please check this [Colab notebook for preparing training data](https://colab.research.google.com/drive/122bYIInseyFwrRFlTNEGLSNFP594i9OV)

Load `short_ans_raw_df.pkl`

In [0]:
short_ans_raw_df = pd.read_pickle("drive/My Drive/short_ans_raw_df.pkl")

In [10]:
short_ans_raw_df = short_ans_raw_df[:80]
print(len(short_ans_raw_df))
short_ans_raw_df.head()

80


,example_id,question_text,long_answer_text,yes_no_answer
0,5655493461695504401,which is the most common use of opt-in e-mail ...,<P> A common example of permission marketing i...,NONE
1,5328212470870865242,how i.met your mother who is the mother,"<P> Tracy McConnell , better known as `` The M...",NONE
2,4435104480114867852,what type of fertilisation takes place in humans,<P> The process of fertilization involves a sp...,NONE
3,5289242154789678439,who had the most wins in the nfl,<P> Active quarterback Tom Brady holds the rec...,NONE
4,-2500044561429484630,who played mantis guardians of the galaxy 2,<P> Pom Klementieff ( born 3 May 1986 ) is a F...,NONE


Load `yes_no_ans_df.pkl`

In [0]:
train_df = pd.read_pickle('drive/My Drive/yes_no_ans_df.pkl')

In [12]:
train_df = train_df[:80]
print(len(train_df))
train_df.head()

80


,label_yes_no,token_ids,segment_ids,mask_ids
0,2,"[2, 56, 25, 14, 127, 757, 275, 16, 17034, 8, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,2,"[2, 184, 31, 9, 5909, 154, 449, 72, 25, 14, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,"[2, 98, 1001, 16, 4270, 8005, 4330, 1384, 209,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,2,"[2, 72, 41, 14, 127, 4041, 19, 14, 4101, 3, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,2,"[2, 72, 257, 169, 3409, 16931, 16, 14, 9358, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [56]:
tf.one_hot(train_df['label_yes_no'][0], 3)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>

Create distributed dataset

In [0]:
def df_to_dataset(dataframe, batch_size, task='classing'):
    dataframe = dataframe.copy()
    if task == 'classing':
      label_yes_no = dataframe.pop('label_yes_no')
      dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), label_yes_no))
    elif task == 'squading':
      label_start_token = dataframe.pop('label_start_token')
      label_end_token = dataframe.pop('label_end_token')
      dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), label_start_token, label_end_token))
    
    dataset = dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

In [0]:
batch_size=16
train_ds = df_to_dataset(train_df, batch_size)
dist_train_ds = tpu_strategy.experimental_distribute_dataset(train_ds)

### Create TFAlbertForSequenceClassification Model

In [0]:
class TFAlbertForSequenceClassification(TFAlbertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super(TFAlbertForSequenceClassification, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.albert = TFAlbertModel(config, name="albert")
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            config.num_labels, kernel_initializer=get_initializer(config.initializer_range), name="classifier"
        )

    def call(self, inputs, **kwargs):
        outputs = self.albert(inputs, **kwargs)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        return outputs  # logits, (hidden_states), (attentions)

In [0]:
def create_model():
    # input layers
    token_ids = keras.Input(shape=(512,), dtype='int32', name='token_ids')
    segment_ids = keras.Input(shape=(512,), dtype='int32', name='segment_ids')
    mask_ids = keras.Input(shape=(512,), dtype='int32', name='mask_ids')

    # bert qa layer
    config = AlbertConfig.from_pretrained('albert-base-v2', num_labels=3)
    albert_qa_layer = TFAlbertForSequenceClassification(config)

    # output layer
    albert_qa_outputs = albert_qa_layer([token_ids, mask_ids, segment_ids])
    logits = albert_qa_outputs[0]
    # create model
    model = keras.Model(inputs=[token_ids, mask_ids, segment_ids], outputs=[logits])
    return model

### TPU Training

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
label_yes_no_train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='label_yes_no_train_accuracy')

In [0]:
# tpu custom training loop
with tpu_strategy.scope():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)

In [0]:
@tf.function
def train_step(dist_inputs):
    # calculate loss and gradient for each replica
    def step_fn(inputs):
        features, label_yes_no = inputs
        one_hot_label = tf.one_hot(label_yes_no, 3)
        one_hot_label_index = tf.argmax(one_hot_label, axis=1)

        with tf.GradientTape() as tape:
            logits = model(features)
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=one_hot_label_index, logits=logits)
            avg_loss = loss / batch_size

        gradients = tape.gradient(avg_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(avg_loss)
        label_yes_no_train_accuracy(one_hot_label_index, logits)

        return avg_loss
    
    # combine loss for all replicas
    per_example_losses = tpu_strategy.experimental_run_v2(step_fn, args=(dist_inputs,))
    sum_loss = tpu_strategy.reduce(tf.distribute.ReduceOp.SUM, per_example_losses, axis=0)
    return sum_loss

In [134]:
with tpu_strategy.scope():
  EPOCHS = 10
  train_loss.reset_states()
  label_yes_no_train_accuracy.reset_states()

  for epoch in range(EPOCHS):
    i = 0
    for inputs in dist_train_ds:
      train_step(inputs)
      i = i + 1
      
      print(f"Epoch {epoch} Batch {i}, Loss: {train_loss.result()}, label_yes_no_train_accuracy: {label_yes_no_train_accuracy.result()*100}")

      train_loss.reset_states()
      label_yes_no_train_accuracy.reset_states()

Epoch 0 Batch 1, Loss: 0.09206603467464447, label_yes_no_train_accuracy: 0.0
Epoch 0 Batch 2, Loss: 0.043868012726306915, label_yes_no_train_accuracy: 100.0
Epoch 0 Batch 3, Loss: 0.020408928394317627, label_yes_no_train_accuracy: 100.0
Epoch 0 Batch 4, Loss: 0.010547973215579987, label_yes_no_train_accuracy: 100.0
Epoch 0 Batch 5, Loss: 0.006406833417713642, label_yes_no_train_accuracy: 100.0
Epoch 1 Batch 1, Loss: 0.004531951621174812, label_yes_no_train_accuracy: 100.0
Epoch 1 Batch 2, Loss: 0.003284827573224902, label_yes_no_train_accuracy: 100.0
Epoch 1 Batch 3, Loss: 0.0025544294621795416, label_yes_no_train_accuracy: 100.0
Epoch 1 Batch 4, Loss: 0.002187750767916441, label_yes_no_train_accuracy: 100.0
Epoch 1 Batch 5, Loss: 0.0020634301472455263, label_yes_no_train_accuracy: 100.0
Epoch 2 Batch 1, Loss: 0.001992674544453621, label_yes_no_train_accuracy: 100.0
Epoch 2 Batch 2, Loss: 0.13394922018051147, label_yes_no_train_accuracy: 50.0
Epoch 2 Batch 3, Loss: 0.1295696496963501, 